In [41]:
# carga doc en la memoria
def load_doc(filename):
 # abrir el archivo como de sólo lectura
 file = open(filename, 'r')
 # leer todo el texto
 text = file.read()
 # cerrar el archivo
 file.close()
 return text
 
# guardar tokens en un archivo, un diálogo por línea
def save_doc(lines, filename):
 data = '\n'.join(lines)
 file = open(filename, 'w')
 file.write(data)
 file.close()
 
# cargar texto
raw_text = load_doc('rhyme.txt')
print(raw_text)
# Limpiar
tokens = raw_text.split()
raw_text = ' '.join(tokens)
# organizarse en secuencias de personajes
length = 11
sequences = list()
for i in range(length, len(raw_text)):
 # seleccionar secuencia de tokens
 seq = raw_text[i-length:i+1]
 # tienda
 sequences.append(seq)
print('Secuencias totales: %d' % len(sequences))
# guardar secuencias en un archivo
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

Un elefante se balanceaba,
sobre la tela de una araña,
Como veía que resistía, 
fue a llamar otro elefante.
Dos elefantes se balanceaban, 
sobre la tela de una araña,
Como veían que resistía, 
fueron a llamar otro elefante
Tres elefantes se balanceaban, 
sobre la tela de una araña,
Como veían que resistía, 
fueron a llamar otro elefante,
Cuatro elefantes se balanceaban, 
sobre la tela de una araña,
Como veían que resistía, 
fueron a llamar otro elefante.
Cinco elefantes se balanceaban, 
sobre la tela de una araña,
Como veían que resistía, 
fueron a llamar otro elefante.
Secuencias totales: 556


In [42]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 
# cargar doc en la memoria
def load_doc(filename):
 # abrir archivo en modo solo lectura
 file = open(filename, 'r')
 # leer todo el texto
 text = file.read()
 # cerrar el archivo
 file.close()
 return text
 
# definir el modelo
def define_model(X):
 model = Sequential()
 model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
 model.add(Dense(vocab_size, activation='softmax'))
 # compilar modelo
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # resumen del modelo definido
 model.summary()
 #plot_model(model, to_file='model.png', show_shapes=True)
 return model
 
# cargar
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')
# enteros codifican secuencias de caracteres
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
 # línea entera de codificación
 encoded_seq = [mapping[char] for char in line]
 # almacenar
 sequences.append(encoded_seq)
# tamaño del vocabulario
vocab_size = len(mapping)
print('Tamaño del vocabulario: %d' % vocab_size)
# separar en entrada y salida
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
# modelo adecuado
y = to_categorical(y, num_classes=vocab_size)
# definir modelo
model = define_model(X)
# Guardar el mapeo
model.fit(X, y, epochs=100, verbose=2)
# guardar el modelo en el archivo
model.save('model.h5')
dump(mapping, open('mapping.pkl', 'wb'))

Tamaño del vocabulario: 27
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 75)                30900     
_________________________________________________________________
dense_6 (Dense)              (None, 27)                2052      
Total params: 32,952
Trainable params: 32,952
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
 - 0s - loss: 3.2372 - accuracy: 0.1673
Epoch 2/100
 - 0s - loss: 2.9252 - accuracy: 0.1547
Epoch 3/100
 - 0s - loss: 2.7216 - accuracy: 0.1709
Epoch 4/100
 - 0s - loss: 2.6575 - accuracy: 0.2068
Epoch 5/100
 - 0s - loss: 2.5883 - accuracy: 0.2392
Epoch 6/100
 - 0s - loss: 2.5150 - accuracy: 0.2482
Epoch 7/100
 - 0s - loss: 2.4329 - accuracy: 0.2500
Epoch 8/100
 - 0s - loss: 2.3466 - accuracy: 0.2572
Epoch 9/100
 - 0s - loss: 2.2573 - accuracy: 0.2824
Epoch 10/100
 - 0

In [43]:
from pickle import load
from numpy import array
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [44]:
# generar una secuencia de caracteres con un modelo de lenguaje
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
 in_text = seed_text
 # generar un número fijo de caracteres
 for _ in range(n_chars):
 # codificar los caracteres como enteros
    encoded = [mapping[char] for char in in_text]
 # cortar secuencias a una longitud fija
 encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
 # una codificacion caliente
 encoded = to_categorical(encoded, num_classes=len(mapping))
 # Predecir el carácter
 yhat = model.predict_classes(encoded, verbose=0)
 # invierte un número entero del mapa a un carácter
 out_char = ''
 for char, index in mapping.items():
  if index == yhat:
   out_char = char
   break
  # añadir a la entrada
  in_text += char
 return in_text

In [45]:
# cargar el modelo
model = load_model('model.h5')
# cargar el mapeo
mapping = load(open('mapping.pkl', 'rb'))
# Poniendo a prueba el comienzo de la rima
print(generate_seq(model, mapping, 11, 'Un elefante', 20))
# prueba en la línea media
print(generate_seq(model, mapping, 11, 'Cinco elefa ', 20))
# prueba no en el original
print(generate_seq(model, mapping, 11, 'Como veía q', 20))

Un elefante

Cinco elefa 
 ,.CDTUabcdefilmnoqrs
Como veía q
 ,.CDTUabcdefilmnoqrst
